ALL THE EXTERNAL REUIQREMENTS ARE PRESNET IN ./requirement.txt

In [ ]:
%pip install -r requirement.txt

In [ ]:
#error can occur, careful!!
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

In [2]:
import os
cwd_path= os.getcwd()
cwd_path= cwd_path.replace("\\","/")

Lets scrape the required dataset 
Input will be a text file containing articles abstracts from pubmed

In [5]:
from my_packages.get_document import module_get_document

kg_disease= str(input("please enter the disease you want a knowledge graph for: "))
no_of_articles= int(input("please enter the number of articles you want to consider: ")) #expected wall time for 10 articles around 12 mins
dataset_file_path= cwd_path + "/datasets/dataset.txt"

module_get_document.get_docx(kg_disease, no_of_articles, dataset_file_path)


Search results saved to a text file.


Lets crete a log file for ouputs 

In [6]:
log_file_path= cwd_path + "/ouput/output_log.txt"

try:
    with open(log_file_path, 'w') as fh:
        fh.write("welcome to the log file!")
except Exception as e:
    print(e)

Lets create a dataframe that can be later deployed to neo4J

In [7]:
from my_packages.get_knowledgeGraph import module_getKnowledgeGrpah

output_df_path= module_getKnowledgeGrpah.get_gph(cwd_path, log_file_path, dataset_file_path)


d:\VIISem\knowledgeGraphUsingMondo\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
100%|██████████| 116/116 [03:31<00:00,  1.82s/it]


Lets deploy df to neo4J :)

In [8]:
from my_packages.create_graph import module_createGraph
#the load_gh fn takes input of bolt url and password of sandbox, create your own sandbow on neo4J
class_labels= module_createGraph.load_gh("bolt://54.152.14.194:7687", "attack-possibilities-workman", output_df_path)

{'total': 112, 'batches': 1, 'time': 4.702183961868286}
{'total': 0, 'batches': 1, 'time': 4.342350244522095}
{'total': 0, 'batches': 1, 'time': 1.8585114479064941}
{'total': 0, 'batches': 1, 'time': 1.2018826007843018}


* Ignore below if you dont need to check efficiency!!

In [4]:
annotated_file_path= cwd_path + "/ouput/annotated.csv"
from my_packages.check_efficiencyGraph import module_get_efficiency
module_get_efficiency.check_ef(output_df_path, annotated_file_path)

NameError: name 'output_df_path' is not defined